In [146]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
import keras
from keras.utils import to_categorical
from keras.constraints import maxnorm
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, PReLU, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [147]:
def LoadData():
    train = pd.read_csv('./data/train.csv')
    test = pd.read_csv('./data/test.csv')
    Y_train = np.array(train['label'])
    Y_train = to_categorical(Y_train)
    
    X = np.array(train['feature'])
    X_train = []
    for i in range(X.shape[0]):
        x = np.array(X[i].split(' '))
        x = x.astype(np.int)
        X_train.append(x)
    X_train = np.array(X_train)
    X_train = X_train.reshape(X_train.shape[0],48,48,1)
    
    X = np.array(test['feature'])
    X_test = []
    for i in range(X.shape[0]):
        x = np.array(X[i].split(' '))
        x = x.astype(np.int)
        X_test.append(x)
    X_test = np.array(X_test)
    X_test = X_test.reshape(X_test.shape[0],48,48,1)
    
    return X_train, Y_train, X_test

In [148]:
labels = {0: "angry", 1:"disgust", 2:"fear", 3:"happy", 4:"sad", 5:"surprise", 6:"neutral"}
X_train, Y_train, X_test = LoadData()

In [151]:
def model_create():
    model = Sequential()
    
    model.add(Conv2D(256, (5, 5), activation='relu', input_shape=(48,48,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    for i in range(3):
        model.add(Conv2D(256, (3, 3), activation='relu',kernel_constraint=maxnorm(5)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
        model.add(BatchNormalization(scale=False, center=False, momentum=0.99, epsilon=0.001))

    model.add(Flatten())
    
    for i in range(2):
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
        
    model.add(Dense(7, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [152]:
outfile  = 'model12'
model = model_create()
checkpoints = ModelCheckpoint('./model/' + outfile  + '.h5', verbose=2, monitor='val_acc', save_best_only=True, mode='auto')

In [ ]:
train = model.fit(X_train, Y_train, batch_size=512, validation_split=0.2, epochs=150, verbose=1,shuffle=True,callbacks=[checkpoints])

Train on 22967 samples, validate on 5742 samples
Epoch 1/150
22967/22967 [==============================] - 10s 440us/step - loss: 0.2980 - acc: 0.9046 - val_loss: 1.5220 - val_acc: 0.6142

Epoch 00001: val_acc did not improve from 0.62522
Epoch 2/150
22967/22967 [==============================] - 9s 374us/step - loss: 0.2838 - acc: 0.9088 - val_loss: 1.5700 - val_acc: 0.6217

Epoch 00002: val_acc did not improve from 0.62522
Epoch 3/150
22967/22967 [==============================] - 9s 373us/step - loss: 0.2884 - acc: 0.9076 - val_loss: 1.5742 - val_acc: 0.6177

Epoch 00003: val_acc did not improve from 0.62522
Epoch 4/150
22967/22967 [==============================] - 9s 374us/step - loss: 0.2758 - acc: 0.9102 - val_loss: 1.5998 - val_acc: 0.6144

Epoch 00004: val_acc did not improve from 0.62522
Epoch 5/150
22967/22967 [==============================] - 9s 372us/step - loss: 0.2785 - acc: 0.9096 - val_loss: 1.5991 - val_acc: 0.6125

Epoch 00005: val_acc did not improve from 0.62522


22967/22967 [==============================] - 9s 375us/step - loss: 0.2055 - acc: 0.9353 - val_loss: 1.7835 - val_acc: 0.6160

Epoch 00043: val_acc did not improve from 0.62644
Epoch 44/150
22967/22967 [==============================] - 9s 371us/step - loss: 0.2079 - acc: 0.9357 - val_loss: 1.7321 - val_acc: 0.6118

Epoch 00044: val_acc did not improve from 0.62644
Epoch 45/150
22967/22967 [==============================] - 9s 374us/step - loss: 0.2092 - acc: 0.9353 - val_loss: 1.7379 - val_acc: 0.6156

Epoch 00045: val_acc did not improve from 0.62644
Epoch 46/150
22967/22967 [==============================] - 9s 375us/step - loss: 0.1996 - acc: 0.9368 - val_loss: 1.8280 - val_acc: 0.6162

Epoch 00046: val_acc did not improve from 0.62644
Epoch 47/150
22967/22967 [==============================] - 9s 375us/step - loss: 0.2066 - acc: 0.9337 - val_loss: 1.7204 - val_acc: 0.6174

Epoch 00047: val_acc did not improve from 0.62644
Epoch 48/150
22967/22967 [==============================] 

In [156]:
model = load_model('./model/' + outfile  + '.h5')
ans = model.predict_classes(X_test).astype('int')
ans = np.column_stack([list(range(X_test.shape[0])), ans])
ans = pd.DataFrame(ans,columns=['id', 'label'])
ans.to_csv('data/'+ outfile +'.csv', index=False)